In [ ]:
# https://pmc.ncbi.nlm.nih.gov/tools/ftp/
# https://ftp.ncbi.nlm.nih.gov/pub/pmc/readme.txt
jlist_url='https://ftp.ncbi.nlm.nih.gov/pub/pmc/jlist.csv'

In [28]:
entrez_journals_url = 'https://ftp.ncbi.nih.gov/pubmed/J_Entrez.txt'

In [ ]:
import fsspec

In [32]:
!pip install aiohttp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [aiohttp]m6/7 [aiohttp]


In [34]:
entrez_lines = fsspec.open(entrez_journals_url).open().readlines()

In [58]:
entrez_lines[:10]

[b'--------------------------------------------------------\n',
 b'JrId: 1\n',
 b"JournalTitle: AADE editors' journal\n",
 b'MedAbbr: AADE Ed J\n',
 b'ISSN (Print): 0160-6999\n',
 b'ISSN (Online): \n',
 b'IsoAbbr: AADE Ed J\n',
 b'NlmId: 7708172\n',
 b'--------------------------------------------------------\n',
 b'JrId: 2\n']

In [76]:
entrez_sel = []
for line in entrez_lines:

    if 'IsoAbbr:' in str(line):
        val = str(line)[11:-3]
        if 'neuro' in val.lower() or 'brain' in val.lower():
            entrez_sel.append(val)
            


In [77]:
len(entrez_sel)

861

In [81]:
entrez_sel[18]

'AJNR Am J Neuroradiol'

In [ ]:
from xml.etree import ElementTree as ET
import requests

def get_pmids(jname, yr=2024, retstart=0):
    # https://www.ncbi.nlm.nih.gov/books/NBK25500/
    searchurl = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term="%s"[ta]+AND+%d[pdat]&retstart=%d&retmax=100'
    
    actualurl = searchurl % (jname, yr, retstart)
    print(actualurl)
    resp = requests.get(actualurl)
    
    root = ET.fromstring(resp.text)
    
    cnt = int(root.findall('.//Count')[0].text)
    
    pmids = []
    for ch in root.findall('.//IdList/'):
        pmids.append(ch.text)
    
    if retstart+100 > cnt: # no more calls
        return pmids
    
    else:
        return pmids + get_pmids(jname, yr, retstart+100)
    


In [88]:
pmids=get_pmids(entrez_sel[18],2024)

https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term="AJNR Am J Neuroradiol"[ta]+AND+2024[pdat]&retstart=0&retmax=100


IndexError: list index out of range

## using pmc

In [35]:
jlist_lines = fsspec.open(jlist_url).open().readlines()

In [36]:
jlist_lines[:5]

[b'Journal title,NLM TA,pISSN,eISSN,Publisher,LOCATORplus ID,Latest issue,Earliest volume,Free access,Open access,Participation level, Deposit status, Journal URL\n',
 b'"3 Biotech","3 Biotech","2190-572X","2190-5738","Springer","101565857","v.15(5);May 2025","v.1;2011","12 months","Some"," Full "," ",http://www.ncbi.nlm.nih.gov/pmc/journals/1811/\n',
 b'"3D Printing and Additive Manufacturing","3D Print Addit Manuf","2329-7662","2329-7670","Mary Ann Liebert, Inc.","101649453","v.12(2);Apr 2025","v.7;2020","12 months","Some"," Full "," ",http://www.ncbi.nlm.nih.gov/pmc/journals/4318/\n',
 b'"3D Printing in Medicine","3D Print Med","","2365-6271","BMC","101721758","v.10;Dec 2024","v.2;2016","Immediate","All"," Full "," ",http://www.ncbi.nlm.nih.gov/pmc/journals/3516/\n',
 b'"AACE Clinical Case Reports","AACE Clin Case Rep","","2376-0605","American Association of Clinical Endocrinology","101670593","v.11(2);Mar-Apr 2025","v.3;2017","Immediate","Some"," Full "," ",http://www.ncbi.nlm.nih.

In [39]:
heading = str(jlist_lines[0]).strip().split(',')
jlist = [str(x).split(',') for x in jlist_lines[1:]]

In [43]:
selected = []
for rec in jlist:
    if 'neuro' in rec[1].lower() or 'brain' in rec[1].lower():
        selected.append(rec[1].strip('"'))

In [44]:
len(selected)

195

In [45]:
selected

['ACS Chem Neurosci',
 'Acta Neuropathol Commun',
 'Aging Brain',
 'AIMS Neurosci',
 'AJNR Am J Neuroradiol',
 'Am J Neurodegener Dis',
 'Ann Clin Transl Neurol',
 'Ann Indian Acad Neurol',
 'Ann Neurosci',
 'Arch Clin Neuropsychol',
 'Arq Neuropsiquiatr',
 'Asian J Neurosurg',
 'ASN Neuro',
 'Audiol Neurootol',
 'Basic Clin Neurosci',
 'Behav Brain Funct',
 'Behav Neurol',
 'BMC Neurol',
 'BMC Neurosci',
 'BMJ Neurol Open',
 'Brain',
 'Brain Behav',
 'Brain Circ',
 'Brain Commun',
 'Brain Connect',
 'Brain Inform',
 'Brain Neurorehabil',
 'Brain Neurosci Adv',
 'Brain Pathol',
 'Brain Plast',
 'Brain Sci',
 'Brain Spine',
 'Brain Tumor Res Treat',
 'Cardiovasc Psychiatry Neurol',
 'Case Rep Neurol',
 'Case Rep Neurol Med',
 'Cell Mol Neurobiol',
 'Child Neurol Open',
 'Chin Neurosurg J',
 'Clin Neuropathol',
 'Clin Neurophysiol Pract',
 'Clin Neuropsychiatry',
 'Clin Psychopharmacol Neurosci',
 'CNS Neurosci Ther',
 'Cogn Neurodyn',
 'Compr Psychoneuroendocrinol',
 'Comput Intell Neur